In [1]:
import polars as pl
import plotly.express as px

In [2]:
schema = {
    "id": pl.UInt32,
    "age": pl.UInt32,
    "job": pl.String,
    "marital": pl.String,
    "education": pl.String,
    "default": pl.String,
    "housing": pl.String,
    "loan": pl.String,
    "contact": pl.String,
    "month": pl.String,
    "day_of_week": pl.String,
    "duration": pl.UInt32,
    "campaign": pl.UInt32,
    "pdays": pl.UInt32,
    "previous": pl.UInt32,
    "poutcome": pl.String,
    "emp.var.rate": pl.Float32,
    "cons.price.idx": pl.Float32,
    "cons.conf.idx": pl.Float32,
    "euribor3m": pl.Float32,
    "nr.employed": pl.Float32, # start with float, cause one value is float
    "y": pl.String,
    "test_control_flag": pl.String
    
}

In [3]:
data = pl.read_csv("data/bank_data_prediction_task_2024.csv", schema=schema, null_values=["NA"])

# change datatypes


In [4]:
#change month to int
data = data.with_columns(pl.col("month")
            .replace({
                "jan": 1,
                "feb": 2,
                "mar": 3,
                "apr": 4,
                "may": 5,
                "jun": 6,
                "jul": 7,
                "aug": 8,
                "sep": 9,
                "oct": 10,
                "nov": 11,
                "dec": 12
            }).cast(pl.UInt32)
        )

In [5]:
#change day of week to int
data = data.with_columns(pl.col("day_of_week")
            .replace({
                "mon": 1,
                "tue": 2,
                "wed": 3,
                "thu": 4,
                "fri": 5
            }).cast(pl.UInt32)
        )

In [6]:
#turn nr.employed to int
data = data.with_columns(pl.col("nr.employed").cast(pl.Int32))

In [7]:
#turn y to 0 and 1
data = data.with_columns(pl.col("y").replace({'no': 0, 'yes': 1}))

In [8]:
data.describe()

statistic,id,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,test_control_flag
str,f64,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,str,str
"""count""",41188.0,41188.0,"""41188""","""41188""","""41188""","""41188""","""41188""","""41188""","""16476""",16476.0,16476.0,16476.0,16476.0,41188.0,41188.0,"""41188""",41188.0,40938.0,41188.0,41188.0,41188.0,"""41188""","""41188"""
"""null_count""",0.0,0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""24712""",24712.0,24712.0,24712.0,24712.0,0.0,0.0,"""0""",0.0,250.0,0.0,0.0,0.0,"""0""","""0"""
"""mean""",20594.5,40.02406,null,null,null,null,null,null,null,6.599539,2.984644,259.660172,2.546856,962.475454,0.172963,null,0.081886,93.575951,-40.502602,3.621291,5166.849446,null,null
"""std""",11890.09578,10.42125,null,null,null,null,null,null,null,2.041805,1.409035,260.660561,2.699664,186.910907,0.494901,null,1.57096,0.57889,4.628198,1.734447,72.328375,null,null
"""min""",1.0,17.0,"""admin.""","""divorced""","""basic.4y""","""no""","""no""","""no""","""cellular""",3.0,1.0,0.0,1.0,0.0,0.0,"""failure""",-3.4,92.200996,-50.799999,0.634,4963.0,"""0""","""campaign group"""
"""25%""",10298.0,32.0,null,null,null,null,null,null,null,5.0,2.0,103.0,1.0,999.0,0.0,null,-1.8,93.074997,-42.700001,1.344,5099.0,null,null
"""50%""",20595.0,38.0,null,null,null,null,null,null,null,6.0,3.0,180.0,2.0,999.0,0.0,null,1.1,93.749001,-41.799999,4.857,5191.0,null,null
"""75%""",30891.0,47.0,null,null,null,null,null,null,null,8.0,4.0,321.0,3.0,999.0,0.0,null,1.4,93.994003,-36.400002,4.961,5228.0,null,null
"""max""",41188.0,98.0,"""unknown""","""unknown""","""unknown""","""yes""","""yes""","""yes""","""telephone""",12.0,5.0,4918.0,43.0,999.0,7.0,"""success""",1.4,94.766998,-26.9,5.045,5228.0,"""1""","""control group"""


# Missing values

we have two places of missing values:
1) for control group all of contact, month, day_of_week, campaign, duration are empty
2) for consumer price index there are 250 missing values. 

In [9]:
#we can see that the observations are rather sorted, hence we can fill the null values with forward fill
data = data.with_columns(pl.col("cons.price.idx").forward_fill())

## Split control and campaign groups, to better deal with missing

In [12]:
data_control = data.filter(pl.col("test_control_flag") == "control group").drop("test_control_flag")
data_campaign = data.filter(pl.col("test_control_flag") == "campaign group").drop("test_control_flag")   

In [13]:
data_control = data_control.drop(["month", "day_of_week", "duration", "contact", "campaign" ])